<a href="https://colab.research.google.com/github/midhunjmes/presidio_final/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install spacy


In [ ]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 34.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd
import spacy


nlp = spacy.load("en_core_web_sm")

def detect_noun(file_path):
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, engine="python")
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

    sensitive = []

    for col in df.columns:
        if df[col].dtype in ['int64', 'float64']:
            continue  # Skip purely numeric columns

        text_samples = df[col].astype(str).head(5)  # Take first 5 values
        noun_count = 0
        total_count = 0

        for value in text_samples:
            if "%" in value or value.replace(".", "").isdigit():
                continue  # Skip percentage or number-like values

            doc = nlp(value)
            for token in doc:
                if token.pos_ in ['NOUN', 'PROPN']:
                    noun_count += 1
                total_count += 1

        # Mark column as sensitive only if a significant portion are nouns
        if total_count > 0 and (noun_count / total_count) > 0.2:
            sensitive.append(col)

    return sensitive





In [ ]:
import pandas as pd
import re

def descriptive_columns(file_path):
    """
    Reads a CSV or Excel file, removes column names similar to 'description', 'remarks', etc.,
    and returns a filtered list of sensitive columns.
    """
    # Define keywords to filter out
    keywords = ["description", "remarks", "notes", "comments", "observations", "details", "summary", "explanation",
    "reviews", "feedback", "testimonials", "opinions", "assessment", "suggestions", "experience",
    "incident_report", "case_notes", "audit_notes", "findings", "status_update", "history", "progress_report",
    "additional_info", "clarifications", "justification", "annotations", "excerpts", "statement", "explanation_text"]

    # Ensure columns are properly loaded from CSV/Excel
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, nrows=1)  # Read only header
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path, nrows=1, engine="openpyxl")  # Read only header
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

    # Get actual column names
    all_columns = df.columns.tolist()


    des=[col for col in all_columns if any(re.search(keyword, col, re.IGNORECASE) for keyword in keywords)]
    return des




In [ ]:
pip install presidio_analyzer presidio_anonymizer flair regex

In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 1.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
from presidio_analyzer import AnalyzerEngine,PatternRecognizer,Pattern,RecognizerResult
from presidio_anonymizer import AnonymizerEngine
from presidio_analyzer.nlp_engine import NlpEngineProvider
import re
from flair.data import Sentence
from flair.nn import Classifier

In [ ]:
ner_tagger = Classifier.load("flair/ner-english-large")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

2025-03-20 06:32:59,579 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [ ]:
import pandas as pd
nlp_config = {
    "nlp_engine_name": "spacy",
    "models": [{"lang_code": "en", "model_name": "en_core_web_lg"}]
}
street_address_pattern = Pattern(
    name="street_address_pattern",
    regex=r"\d+\s[A-Za-z]+\s(?:Street|St|Avenue|Ave|Road|Rd|Boulevard|Blvd|Lane|Ln|Drive|Dr|Court|Ct|Place|Pl|Square|Sq)\b",
    score=0.85
)
street_recognizer = PatternRecognizer(
    supported_entity="STREET_ADDRESS",
    patterns=[street_address_pattern]
)



provider = NlpEngineProvider(nlp_configuration=nlp_config)
nlp_engine = provider.create_engine()

analyzer=AnalyzerEngine(nlp_engine=nlp_engine)
anonymizer=AnonymizerEngine()
analyzer.registry.add_recognizer(street_recognizer)

def detect_locations_with_flair(text):
    sentence = Sentence(text)
    ner_tagger.predict(sentence)

    location_entities = []
    for entity in sentence.get_spans("ner"):
        if entity.tag in ["LOC","ORG", 'PER']:
          entity_type = "LOCATION" if entity.tag == "LOC" else "ORGANIZATION" if entity.tag == "ORG" else "PERSON"
          location_entities.append(RecognizerResult(
                entity_type=entity_type,
                start=entity.start_position,
                end=entity.end_position,
                score=entity.score
            ))
    return location_entities

def analyze_and_anonymize(text):
    presidio_results = analyzer.analyze(
        text=text,
        entities=["PERSON","EMAIL_ADDRESS", "PHONE_NUMBER", "STREET_ADDRESS"],
        language="en",
    )

    flair_results = detect_locations_with_flair(text)
    all_results =  flair_results + presidio_results

    anonymized_text = anonymizer.anonymize(text, all_results).text
    return anonymized_text



In [ ]:
file_path="first_100_rows.csv"
sensitive=detect_noun(file_path)
print(sensitive)
descriptive_columns(file_path)

['name', 'reviews', 'description', 'ceo_approval', 'ceo_count', 'salary', 'interview_duration', 'interview_count', 'headquarters', 'industry', 'revenue', 'website']


['reviews', 'description', 'interview_experience']

In [ ]:
import openpyxl
import json
mapping={}
def excel_an(input_file, output_file):
    wb = openpyxl.load_workbook(input_file)
    sensitive_old =detect_noun(input_file)
    desc=descriptive_columns(file_path,sensitive_old)
    sensitive = list(set(sensitive_old) - set(desc))
    #______________________________________________________________________________________________
    for sheet_name in wb.sheetnames:
        sheet = wb[sheet_name]
        headers = [cell.value for cell in sheet[1]]  # Get column names

        column_counters = {header: 1 for header in headers if header in sensitive}  # Initialize counters for each column

        for row in sheet.iter_rows(min_row=2):
            for idx, cell in enumerate(row):
                column_name = headers[idx]
                if column_name in sensitive:
                    if cell.value and isinstance(cell.value, str):
                        original_value = cell.value
                        anonymized_value = f"{column_name}{column_counters[column_name]}"
                        cell.value = anonymized_value
                        column_counters[column_name] += 1  # Increment counter
                        mapping[anonymized_value] = original_value
    #______________________________________________________________________________________________
    for sheet_name in wb.sheetnames:
        sheet = wb[sheet_name]
        headers = [cell.value for cell in sheet[1]]  # Get column names
        for header in headers:
            if header in desc:
                for row in sheet.iter_rows(min_row=2):
                    for cell in row:
                        if isinstance(cell.value, str):
                            cell.value = detect_noun_desc(cell.value)



    with open("mappings.json", "w") as f:
        json.dump(mapping, f)

    wb.save(output_file)
    print(f"Anonymized file saved as {output_file}")

In [ ]:
# excel_an("test1.xlsx","pres_an_2.xlsx")

In [ ]:
import pandas as pd
import json

mapping = {}

def csv_an(input_file, output_file):
    df = pd.read_csv(input_file, engine="python")

    sensitive_old = detect_noun(input_file)
    desc = descriptive_columns(input_file)
    sensitive = list(set(sensitive_old) - set(desc))

    column_counters = {col: 1 for col in df.columns if col in sensitive_old}
    column_mappings = {col: {} for col in sensitive}  # Store mappings for each column

    # Anonymize sensitive columns while maintaining consistency
    for col in sensitive:
        new_values = []
        for val in df[col].astype(str):
            if pd.notna(val):
                if val in column_mappings[col]:
                    anonymized_value = column_mappings[col][val]  # Use existing mapping
                else:
                    anonymized_value = f"{col}{column_counters[col]}"
                    column_mappings[col][val] = anonymized_value  # Store new mapping
                    column_counters[col] += 1  # Increment counter

                mapping[anonymized_value] = val
                new_values.append(anonymized_value)
            else:
                new_values.append(val)

        df[col] = new_values

    # Anonymize descriptive columns
    for col in desc:
        for idx, val in enumerate(df[col].astype(str)):
            if pd.notna(val):
                an_values = detect_noun_desc(val)
                df.at[idx, col] = an_values
                mapping[an_values] = val

    df.to_csv(output_file, index=False)
    print(f"✅ Anonymized file saved as {output_file}")

    # Save mapping as JSON
    with open("mappings.json", "w") as f:
        json.dump(mapping, f)



In [ ]:
input_csv = "first_100_rows.csv"
output_csv = "anonymized_file_1.csv"
csv_an(input_csv, output_csv)

Anonymized file saved as anonymized_file_1.csv


In [ ]:

nlp=spacy.load("en_core_web_sm")
def detect_noun_desc(text):
    doc = nlp(text)
    modified_text=[]
    for token in doc:
        if token.pos_ in ['PROPN']:
            modified_text.append("<sensitive>")
        else:
            modified_text.append(token.text)
    text=" ".join(modified_text)
    return text


In [ ]:
import pandas as pd
import json

def csv_dean(input_file, output_file, mapping_file):
    print("🔄 Loading data...")

    with open(mapping_file, "r") as f:
        mapping = json.load(f)
    df = pd.read_csv(input_file, engine="python", dtype=str)  # Read as string for safety
    mapping_keys = set(mapping.keys())
    df = df.applymap(lambda x: mapping[x] if x in mapping_keys else x)
    df.to_csv(output_file, index=False)

    print(f"✅ De-anonymized file saved as {output_file}")


In [ ]:
csv_an("testing2.csv","intermediate.csv")


✅ Anonymized file saved as intermediate.csv


In [ ]:
csv_dean("intermediate.csv","de_an_1.csv","mappings.json")

🔄 Loading data...
✅ De-anonymized file saved as de_an_1.csv


<ipython-input-54-65e9847d78b3>:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: mapping[x] if x in mapping_keys else x)


In [ ]:
import pandas as pd

def compare_csv(file1, file2, ignore_index=True, columns=None):
    """
    Compare two CSV files and check if they are identical.

    :param file1: Path to the first CSV file.
    :param file2: Path to the second CSV file.
    :param ignore_index: If True, ignores index differences.
    :param columns: List of columns to compare (default: all).
    :return: True if files are identical, else False and shows differences.
    """

    # Load CSV files
    df1 = pd.read_csv(file1)  # Read as string to avoid type mismatches
    df2 = pd.read_csv(file2)

    # Select only specific columns if provided
    if columns:
        df1 = df1[columns]
        df2 = df2[columns]

    # Ignore index differences if required
    if ignore_index:
        df1 = df1.reset_index(drop=True)
        df2 = df2.reset_index(drop=True)

    # Compare DataFrames
    if df1.equals(df2):
        print("✅ The CSV files are identical.")
        return True
    else:
        print("❌ The CSV files are different.")

        # Show differences
        diff = df1.compare(df2)
        print("\n🔍 Differences Found:\n", diff)

        return False

# Example Usage
compare_csv("testing2.csv","de_an_1.csv")


✅ The CSV files are identical.


True

In [ ]:

text='''Founded in 1911, name1 has established itself in the industry1 sector, growing to a massive workforce of 716,906 employees. Operating from locality1, country1, the company maintains a strong presence with over 274,047 current employees. Similarly, name2, founded in 1968, has become a key player in industry2, employing 341,369 individuals globally, with its headquarters in locality2, country2. Another major organization, name3, founded in 1989, has expanded significantly in industry3, boasting a workforce of 455,768 employees. With roots tracing back to 1800, name4 has remained a pillar in industry4, currently employing 445,958 individuals. Lastly, name5, also founded in 1989, has continued to thrive in industry5, with a total employee estimate of 428,960. These companies, each with a vast global workforce, continue to shape their respective industries, leveraging their expertise and experience to drive innovation and growth.'''
print(text)

Founded in 1911, name1 has established itself in the industry1 sector, growing to a massive workforce of 716,906 employees. Operating from locality1, country1, the company maintains a strong presence with over 274,047 current employees. Similarly, name2, founded in 1968, has become a key player in industry2, employing 341,369 individuals globally, with its headquarters in locality2, country2. Another major organization, name3, founded in 1989, has expanded significantly in industry3, boasting a workforce of 455,768 employees. With roots tracing back to 1800, name4 has remained a pillar in industry4, currently employing 445,958 individuals. Lastly, name5, also founded in 1989, has continued to thrive in industry5, with a total employee estimate of 428,960. These companies, each with a vast global workforce, continue to shape their respective industries, leveraging their expertise and experience to drive innovation and growth.


In [ ]:
def dean_text(text,mapping_file):
  with open(mapping_file, "r") as f:
    mapping = json.load(f)
  for key,value in mapping.items():
    text=text.replace(key,value)
  return text

In [ ]:
dean_text(text,'mappings.json')

'Founded in 1911, ibm has established itself in the information technology and services sector, growing to a massive workforce of 716,906 employees. Operating from new york, new york, united states, united states, the company maintains a strong presence with over 274,047 current employees. Similarly, tata consultancy services, founded in 1968, has become a key player in information technology and services, employing 341,369 individuals globally, with its headquarters in bombay, maharashtra, india, india. Another major organization, accenture, founded in 1989, has expanded significantly in information technology and services, boasting a workforce of 455,768 employees. With roots tracing back to 1800, us army has remained a pillar in military, currently employing 445,958 individuals. Lastly, ey, also founded in 1989, has continued to thrive in accounting, with a total employee estimate of 428,960. These companies, each with a vast global workforce, continue to shape their respective indu